# Setup

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

from torchmetrics import Accuracy
from torch.optim import Adam
from torch.optim.lr_scheduler import CosineAnnealingLR

from sklearn.metrics import classification_report
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import roc_auc_score
from sklearn import metrics

In [2]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

import random
from glob import glob
from tqdm import tqdm
from scipy.io import loadmat

import torch
from torch import nn
from torch.optim import Adam
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F

from datetime import datetime

In [3]:
from IPython.display import clear_output
import os, sys, shutil
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from tqdm import tqdm, trange
from glob import glob
import random
import cv2 as cv
import torch.nn.functional as F
import json
from sklearn.preprocessing import LabelEncoder

In [4]:
# main_data_dir =  "/media/mountHDD2"
print(os.getcwd())
save_dir = os.getcwd()
os.chdir("..")
# os.chdir("..")
main_dir = os.getcwd() 
print(os.listdir(main_dir))
print(save_dir)

/media/mountHDD2/linh/git/HeartResearch/Chapman Dataset/Reimplement
['Reimplement', 'Data_Explore', 'Diagnostics.xlsx', '.ipynb_checkpoints', '2D_model', '1D_Model', '2D_Loss']
/media/mountHDD2/linh/git/HeartResearch/Chapman Dataset/Reimplement


In [5]:
data_dir = "/media/mountHDD3/data_storage/biomedical_data/ecg_data/ECGDataDenoised"
label_file = main_dir + "/Diagnostics.xlsx"

In [6]:
ls = []
for file in glob(data_dir + "/*"):
    ls.append(file.split("/")[-1].split(".")[0].split("_")[0])
ls_list = pd.Series(ls)
ls_list.value_counts()

MUSE    10588
dtype: int64

In [7]:
diag_df = pd.read_excel(label_file)
label_df = diag_df[['FileName', 'Rhythm']]
label_df

,FileName,Rhythm
0,MUSE_20180120_121711_19000,SA
1,MUSE_20180120_121704_86000,SA
2,MUSE_20180113_125357_13000,SA
3,MUSE_20180113_134825_04000,SB
4,MUSE_20180115_123455_79000,SB
...,...,...
10641,MUSE_20181222_204246_47000,SVT
10642,MUSE_20180115_120332_79000,SA
10643,MUSE_20180712_152507_30000,AF
10644,MUSE_20180118_181350_17000,SA


In [8]:
# attribute_df = diag_df.iloc[:, 5:]
attribute_df = diag_df[['FileName', 'VentricularRate', 'AtrialRate', 'QRSDuration', 'QTInterval',
                       'QTCorrected', 'RAxis', 'TAxis', 'QRSCount', 'QOnset', 'QOffset', 'TOffset']]
attribute_df

,FileName,VentricularRate,AtrialRate,QRSDuration,QTInterval,QTCorrected,RAxis,TAxis,QRSCount,QOnset,QOffset,TOffset
0,MUSE_20180120_121711_19000,77,77,80,366,414,59,28,13,211,251,394
1,MUSE_20180120_121704_86000,68,68,80,402,427,29,53,11,220,260,421
2,MUSE_20180113_125357_13000,94,94,86,334,417,86,65,15,213,256,380
3,MUSE_20180113_134825_04000,58,58,102,432,424,2,60,10,211,262,427
4,MUSE_20180115_123455_79000,59,59,84,424,419,55,44,9,221,263,433
...,...,...,...,...,...,...,...,...,...,...,...,...
10641,MUSE_20181222_204246_47000,184,0,234,354,619,178,144,30,159,276,336
10642,MUSE_20180115_120332_79000,69,69,76,400,428,77,63,12,219,257,419
10643,MUSE_20180712_152507_30000,102,340,90,398,518,48,210,17,211,256,410
10644,MUSE_20180118_181350_17000,77,77,102,398,450,-16,30,13,210,261,409


In [9]:
unique_values = np.unique(label_df["Rhythm"].values.tolist())
print(unique_values)

['AF' 'AFIB' 'AT' 'AVNRT' 'AVRT' 'SA' 'SAAWR' 'SB' 'SR' 'ST' 'SVT']


In [10]:
# data_paths = []
# for file in glob(data_dir +"/*"):
#     data_paths.append(file)

In [11]:
# data_dict = {
#     idx : [] for idx in unique_values
# }

# for data_path in data_paths:
#     filename = data_path.split("/")[-1].split(".")[0]
    
#     # Get the corresponding 'Rhythm' values for the filename
#     rhythm_values = label_df[label_df["FileName"] == filename]["Rhythm"].values
    
#     if rhythm_values.size == 1:
#         _cls = rhythm_values.item()  # Safe to use .item() if there's exactly one match
#     elif rhythm_values.size > 1:
#         _cls = rhythm_values[0]  # Take the first element if there are multiple matches
#     else:
#         continue  # Skip if no matches are found
    
#     data_dict[_cls].append(data_path)

# for key in data_dict:
#     print(f"{key}->{len(data_dict[key])}")

In [12]:
label_encoder = LabelEncoder()
label_df['Rhythm'] = label_encoder.fit_transform(label_df['Rhythm'])
print(label_df)

                         FileName  Rhythm
0      MUSE_20180120_121711_19000       5
1      MUSE_20180120_121704_86000       5
2      MUSE_20180113_125357_13000       5
3      MUSE_20180113_134825_04000       7
4      MUSE_20180115_123455_79000       7
...                           ...     ...
10641  MUSE_20181222_204246_47000      10
10642  MUSE_20180115_120332_79000       5
10643  MUSE_20180712_152507_30000       0
10644  MUSE_20180118_181350_17000       5
10645  MUSE_20180116_121646_28000       9

[10646 rows x 2 columns]


/tmp/ipykernel_823169/787631178.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  label_df['Rhythm'] = label_encoder.fit_transform(label_df['Rhythm'])


In [13]:
filenames = label_df["FileName"].values.tolist()

In [14]:
unique_values = np.unique(label_df["Rhythm"].values.tolist())
print(unique_values)

[ 0  1  2  3  4  5  6  7  8  9 10]


In [15]:
data_paths = []
for file in glob(data_dir +"/*"):
    data_paths.append(file)
print(len(data_paths))

10588


In [16]:
ratio = [0.9, 0.1]

train_index = int(len(data_paths)*ratio[0])
valid_index = int(len(data_paths)*(ratio[0]+ratio[1]))
print(train_index)
train_mat_paths = data_paths[:train_index]
valid_mat_paths = data_paths[train_index:valid_index]
print(len(train_mat_paths))

train_label = label_df.iloc[:train_index,:]
valid_label = label_df.iloc[train_index:valid_index,:]

9529
9529


In [17]:
print(len(train_label))
print(len(valid_label))

9529
1059


In [18]:
label_df[label_df["FileName"] == filenames]["Rhythm"].values

array([5, 5, 5, ..., 0, 5, 9])

In [19]:
ECG_feature = attribute_df[attribute_df["FileName"] == 'MUSE_20180120_121711_19000'].iloc[:, 1:].to_numpy()[0]
print(ECG_feature)

[ 77  77  80 366 414  59  28  13 211 251 394]


 # Data Loader

In [20]:
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from torch.nn.functional import one_hot

In [21]:
# data = []
class HeartData(Dataset):
    def __init__(self, data_paths, label_df, attribute_df):
        self.data_paths = data_paths
        random.shuffle(self.data_paths)
        self.label_df = label_df
        self.attribute_df = attribute_df
        normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])
        self.transform = transforms.Compose([
            transforms.Resize((224, 224), antialias=None),
            normalize
        ])

    def __getitem__(self, idx):
        data_path = self.data_paths[idx]        
        # data = loadmat(data_path)['ECG'][0][0][2]
        data = pd.read_csv(data_path, header = None)
        data = data.values.T
        clip_data = data[:, 500:3000]
        clip_data = torch.tensor(clip_data, dtype=torch.float32)
        # normalized_data = (clip_data - clip_data.min()) / (clip_data.max() - clip_data.min())
        # grayscale_images = (normalized_data * 255)
        # grayscale_images = grayscale_images.unsqueeze(0).unsqueeze(0) # (1, 1, h, w)
        # resized_images = F.interpolate(grayscale_images, size=(9*4,2500), mode='bilinear', align_corners=True)
        # resized_images = resized_images.squeeze(0).squeeze(0)
        # torch_data = resized_images.unsqueeze(0).repeat(3, 1, 1)
        # torch_data_resize = self.transform(torch_data)

        filename = data_path.split("/")[-1].split(".")[0]
        label = self.label_df[self.label_df["FileName"] == filename]["Rhythm"].values.item()
        ECG_feature = self.attribute_df[self.attribute_df["FileName"] == filename].iloc[:, 1:].to_numpy()[0]

        return clip_data, label, ECG_feature

    def __len__(self):
        return len(self.data_paths)    

In [22]:
train_ds = HeartData(train_mat_paths, label_df, attribute_df)
valid_ds = HeartData(valid_mat_paths, label_df, attribute_df)

In [23]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu", index = 1)
batch_size = 64

traindl = DataLoader(
    train_ds,
    batch_size=batch_size, 
    shuffle=True, 
    pin_memory=True, 
    num_workers=os.cpu_count()//2
)

validdl = DataLoader(
    valid_ds,
    batch_size=1, 
    # shuffle=True, 
    pin_memory=True, 
    num_workers=os.cpu_count()//2
)

# Model 

In [24]:
import torch
import torch.nn as nn

class DNN(nn.Module):
    def __init__(self):
        super(DNN, self).__init__()
        # Input size: (batch_size, 12, 300)

        self.conv1 = nn.Conv1d(12, 64, kernel_size=21, stride=11)
        # self.pca1 = nn.Linear(64,11)
        self.layer1 = nn.Sequential(
            nn.MaxPool1d(2),
            nn.BatchNorm1d(64),
            nn.LeakyReLU(),
            nn.Dropout(0.3)
        )
         

        self.conv6 = nn.Conv1d(64, 128, kernel_size=7, stride=1)
        # self.pca6 = nn.Linear(128,11)
        self.layer2 = nn.Sequential(
            nn.MaxPool1d(2),
            nn.BatchNorm1d(128),
            nn.Conv1d(128, 256, kernel_size=5, stride=1),
            nn.MaxPool1d(2)
        )
        
        self.conv11 = nn.Conv1d(256, 512, kernel_size=13, stride=1)
        # self.pca11 = nn.Linear(512,11)
        self.layer3 = nn.Sequential(
            nn.ConvTranspose1d(512, 256, kernel_size=7, stride=1),
            nn.Dropout(0.3)
        )

        self.conv14 = nn.Conv1d(256, 256, kernel_size=9, stride=1)
        self.pca14 = nn.Linear(256,11)
        self.pool4 = nn.MaxPool1d(2)

        self.lstm = nn.LSTM(input_size=256, hidden_size=128, batch_first=True)
        self.fc = nn.Linear(128, 11)

    def forward(self, x):
        # Input: (batch_size, 12, 300)
        x = self.conv1(x)  # (batch_size, 64, ?)
        x = self.layer1(x)  # (batch_size, 64, ?)
    
        x = self.conv6(x)  # (batch_size, 128, ?)
        x = self.layer2(x)  # (batch_size, 256, ?)
    
        x = self.conv11(x)  # (batch_size, 512, ?)
        x = self.layer3(x)  # (batch_size, 256, ?)
    
        x = self.conv14(x)  # (batch_size, 256, ?)
        x = self.pool4(x)
    
        # Flatten for Linear layer
        x = x.permute(0, 2, 1)  # (batch_size, seq_len, 256)
        # pca14 = x.reshape(-1, 256)  # (batch_size * seq_len, 256)
        # pca14 = self.pca14(x)  # Linear layer: (batch_size * seq_len, 11)
        
        # # Reshape back to batch format (if needed for downstream tasks)
        # seq_len = pca14.size(0) // x.size(0)  # Compute seq_len
        # pca14 = pca14.view(-1, seq_len, 11)  # (batch_size, seq_len, 11)
    
        # Use pooled feature maps
        # x = self.pool4(x)  # If further processing is required
        # x = x.permute(0, 2, 1)  # (batch_size, seq_len, 256)
    
        x, _ = self.lstm(x)  # LSTM expects (batch_size, seq_len, input_size)
        x = x[:, -1, :]  # Take the last time step's output
        out = self.fc(x)  # Final output layer
        return out

In [25]:
a = torch.rand(1,12,2500)
out_DNN = DNN()(a)
print(out_DNN.shape)

torch.Size([1, 11])


# Training

In [26]:
epoch = 24
lr = 0.001 # lr = 0.001 Acc: 0.821875 lr = 0.0005 Acc: 0.825 Focal_loss Acc: 0.828
best_acc = 0
best_ep = 0

model = DNN()
model.to(device)
optimizer = Adam(model.parameters(), lr=lr)
scheduler = CosineAnnealingLR(optimizer=optimizer, T_max=epoch*len(traindl))

In [27]:
class FocalClassifierV0(nn.Module):
    def __init__(self, gamma=0.3):
        super().__init__()
        
        self.gamma = gamma
        self.act = nn.LogSoftmax(dim=1)
    
    def forward(self, pred, target):

        logits = self.act(pred)

        B, C = tuple(logits.size())

        entropy = torch.pow(1 - logits, self.gamma) * logits * F.one_hot(target, num_classes=C).float()

        return (-1 / B) * torch.sum(entropy)

focalloss_fn = FocalClassifierV0()
loss_fn = nn.CrossEntropyLoss()
# checkpoint_folder = "run_efficientB0_heatmap_gamma0.5_lr0.0005"
checkpoint_folder = "run_DNN_gamma0.3_0.05_lr0001"
# checkpoint_folder = "run_proposed_gamma5_0.01_lr0001_10"

In [28]:
def loss_plot(train_losses, val_losses, n_epochs, check_folder):
    now = datetime.now().strftime("%m-%d-%Y - %H-%M-%S")
    run_dir = save_dir + f"/{check_folder}"
    if not os.path.exists(run_dir):
        os.mkdir(run_dir)
    save_loss_dir = run_dir + "/save_losses"
    if not os.path.exists(save_loss_dir):
        os.mkdir(save_loss_dir)
    save_fig_losses = os.path.join(save_loss_dir, f"plot_losses_epoch{n_epochs}_{now}.png")  

    plt.figure()
    plt.plot(range(epoch), np.array(train_losses), label='Train Loss')
    plt.plot(range(epoch), np.array(val_losses), label='Test Loss')
    plt.xlabel('Epoch')
    # plt.xticks()
    plt.ylabel('Loss')
    plt.title('Training and Test Loss')
    plt.legend()
    plt.savefig(save_fig_losses)
    

def acc_plot(train_cls_acc, val_cls_acc, n_epochs, check_folder):
    now = datetime.now().strftime("%m-%d-%Y - %H-%M-%S")
    run_dir = save_dir + f"/{check_folder}"
    if not os.path.exists(run_dir):
        os.mkdir(run_dir)
#         save_dir = run_dir + f"/{now}"
#         if not os.path.exists(save_dir):
#             os.mkdir(save_dir)
    save_acc_dir = run_dir + "/save_acc"
    if not os.path.exists(save_acc_dir):
        os.mkdir(save_acc_dir)
    save_fig_acc = os.path.join(save_acc_dir, 'plot_acc_epoch{}_{}.png'.format(n_epochs, now))  

    plt.figure()
    plt.plot(range(epoch), np.array(train_cls_acc), label='Train Accuracy')
    plt.plot(range(epoch), np.array(val_cls_acc), label='Test Accuracy')
    plt.xlabel('Epoch')
    # plt.xticks
    plt.ylabel('Accuracy')
    plt.title('Training and Test Accuracy')
    plt.legend()
    plt.savefig(save_fig_acc)

def checkpoint(valid_class_acc, 
               val_total_loss,
               old_valid_class_acc,
               old_valid_loss,
               epoch, 
               model,
               optimizer,
               check_folder
#                    logs
              ):

    if valid_class_acc >= old_valid_class_acc and val_total_loss <= old_valid_loss:
        old_valid_class_acc = valid_class_acc
        old_valid_loss = val_total_loss
        save_dict = {
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': val_total_loss,
            'test_acc': valid_class_acc
        }

     # Saving best model
        now = datetime.now().strftime("%m-%d-%Y - %H-%M-%S")
        run_dir = save_dir + f"/{check_folder}"
        if not os.path.exists(run_dir):
            os.mkdir(run_dir)
    #         save_dir = run_dir + f"/{now}"
    #         if not os.path.exists(save_dir):
    #             os.mkdir(save_dir)
        save_best_model_dir = run_dir + "/save_best_model"
        if not os.path.exists(save_best_model_dir):
            os.mkdir(save_best_model_dir)
        save_best_model_path = save_best_model_dir + f"/{save_dict['loss']:>7f}_{save_dict['test_acc']:>7f}_{now}.pt"
        torch.save(save_dict, save_best_model_path)
        
def classification_report_csv(report, auc, check_folder):
    now = datetime.now().strftime("%m-%d-%Y - %H-%M-%S")
    run_dir = save_dir + f"/{check_folder}"
    if not os.path.exists(run_dir):
        os.mkdir(run_dir)
    save_report_dir = run_dir + "/save_classification_report"
    if not os.path.exists(save_report_dir):
        os.mkdir(save_report_dir)
        
    report_data = report['macro avg']
    del report_data['support']
    report_data.update({'auc': auc})
#     print(type(report_data))
#     print(report_data)

#     dataframe = pd.DataFrame.from_dict(report_data, orient='index')
#     save_report_file = save_report_dir + f"/classification_report_{now}.csv"
#     dataframe.to_csv(save_report_file, index = False)
    with open(save_report_dir + f"/cls_report_{now}.json", "w") as outfile: 
        json.dump(report_data, outfile)
    
def acc_loss_json(log_dict, check_folder):
    now = datetime.now().strftime("%m-%d-%Y - %H-%M-%S")
    run_dir = save_dir + f"/{check_folder}"
    if not os.path.exists(run_dir):
        os.mkdir(run_dir)
    save_json_dir = run_dir + "/save_acc_loss_json"
    if not os.path.exists(save_json_dir):
        os.mkdir(save_json_dir) 
    save_acc_loss_file = save_json_dir + f"/acc_loss_{now}.json"
    with open(save_acc_loss_file, "w") as outfile: 
        json.dump(log_dict, outfile)

In [29]:
log_dict = {
    "train": {
        "acc": [],
        "loss": []
    },
    "valid": {
        "acc": [],
        "loss": []
    }
}

In [30]:
# Training

# class_la = np.unique(label_df["Rhythm"].values.tolist())
class_la = []
for i in range (11):
    class_la.append(i)
for i in range (len(class_la)):
    class_la[i] = str(class_la[i])
train_losses = []
train_acc_plot = []
val_losses = []
val_acc_plot = []
old_valid_class_acc = 0
old_valid_loss = 1e23
for e in range(epoch):
    model.train()
    print(f"Epoch: {e+1}")
    batch_cnt = 0
    total_loss = 0
    correct = 0
    for batch, (train_sig, train_label, _) in tqdm(enumerate(traindl)):
        batch_cnt = batch
        train_sig = train_sig.to(device)
        train_label = train_label.to(device)
        # print("Hi")
        # print(train_sig)
        
        pred = model(train_sig)
        # print("Hello")
        # print(pred)
        
        loss = focalloss_fn(pred, train_label)
        # loss = loss_fn(pred, train_label)
        # print(train_label)
        # print(pred)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        scheduler.step()
        total_loss += loss.item()
        correct += (pred.argmax(1) == train_label).type(torch.float).sum().item()
        
    total_loss /= batch_cnt
    correct /= len(traindl.dataset)
    log_dict["train"]["loss"].append(total_loss)
    log_dict["train"]["acc"].append(correct)
    
    print(f"train loss: {total_loss} - train acc: {100*correct}")

# Valid
    batch_cnt = 0
    val_total_loss = 0
    val_correct = 0
    model.eval()
    y_true_list = [] 
    pred_list = []
    with torch.no_grad():
        for batch, (valid_sig, valid_label, _) in tqdm(enumerate(validdl)):
            batch_cnt = batch
            valid_label = valid_label.to(device)
            
            valid_sig = valid_sig.to(device)  
            pred = model(valid_sig)
            pred_pos = pred.argmax(1)
            
            y_true_list.append(valid_label)
            pred_list.append(pred_pos)
            
            loss = loss_fn(pred, valid_label)
            val_total_loss += loss.item()
            val_correct += (pred.argmax(1) == valid_label).type(torch.float).sum().item()
            
        val_total_loss /= batch_cnt
        val_correct /= len(validdl.dataset)
        log_dict["valid"]["loss"].append(val_total_loss)
        log_dict["valid"]["acc"].append(val_correct)

        if val_correct > best_acc:
            best_acc = val_correct
            best_ep = e 

        print(f"valid loss: {val_total_loss} - valid acc: {100*val_correct}")
#         checkpoint(valid_class_acc = val_correct, 
#                    val_total_loss = val_total_loss,
#                    old_valid_class_acc = old_valid_class_acc ,
#                    old_valid_loss = old_valid_loss,
#                    epoch = e, 
#                    model = model,
#                    optimizer = optimizer,
#                    check_folder = checkpoint_folder)
        if val_correct >= old_valid_class_acc:
            old_valid_class_acc = val_correct
            old_valid_loss = val_total_loss
            save_dict = {
                'epoch': e,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'loss': val_total_loss,
                'test_acc': val_correct
            }

         # Saving best model
            now = datetime.now().strftime("%m-%d-%Y - %H-%M-%S")
            run_dir = save_dir + f"/{checkpoint_folder}"
            if not os.path.exists(run_dir):
                os.mkdir(run_dir)
            save_best_model_dir = run_dir + "/save_best_model"
            if not os.path.exists(save_best_model_dir):
                os.mkdir(save_best_model_dir)
            save_best_model_path = save_best_model_dir + f"/{save_dict['loss']:>7f}_{save_dict['test_acc']:>7f}_{now}.pt"
            torch.save(save_dict, save_best_model_path)
        
print(f"Best acuracy: {best_acc} at epoch {best_ep}")

y_true = torch.cat(y_true_list).cpu().numpy()
pred = torch.cat(pred_list).cpu().numpy()
# print(type(y_true))
# print(y_true)
# print(f"pred: {pred}")

# Save loss and acc to json file
acc_loss_json(log_dict, check_folder = checkpoint_folder)
    
# Save plot acc and loss
# loss_plot(train_losses = train_losses, 
#           val_losses = val_losses, n_epochs = e , 
#           check_folder = checkpoint_folder)
# acc_plot(train_cls_acc = train_acc_plot , val_cls_acc = val_acc_plot, 
#                   n_epochs = e, check_folder = checkpoint_folder)

# AUC
fpr, tpr, thresholds = metrics.roc_curve(y_true, pred, pos_label = 0)
# print(fpr)
# print(f"tpr: {tpr}")
auc1 = metrics.auc(fpr, tpr)

# print(f"hihi {auc1}")
# print(type(auc1))

# Classification report
# for i in range (len(class_la)):
#     class_la[i] = str(class_la[i])

reports = classification_report(y_true, pred, output_dict=True) 
# print(reports)
classification_report_csv(report = reports, auc = auc1, check_folder = checkpoint_folder)

# json_object = json.dumps(reports, indent = 4) 
# print(json_object)
# ROC Curve
# fpr = [0] * 9
# tpr = [0] * 9
# thresholds = [0] * 9
# auc_score = [0] * 9
 
# for i in range(9):
#     fpr[i], tpr[i], thresholds[i] = roc_curve(y_true[:, i],
#                                               pred[:, i])
#     auc_score[i] = auc(fpr[i], tpr[i])
# print(auc_score)
# for i in range(9):
#     r2 = roc_auc_score(y_true, pred, average='micro', multi_class='ovr')
#     print("The ROC AUC score of "+ class_la[i] +" is: "+str(r2))

                    # Compute ROC curve and ROC area for each class
# class_fpr = {}
# class_tpr = {}
# class_roc_auc = dict()
# for i in range(9):
#     class_fpr[i], class_tpr[i], _ = roc_curve(valid_label[:, i].cpu(), pred[:, i].cpu(), drop_intermediate=False)
#     roc_auc[i] = auc(class_fpr[i], class_tpr[i])

# now = datetime.now().strftime("%m-%d-%Y - %H-%M-%S")
# run_dir = save_dir + f"/{check_folder}"
# if not os.path.exists(run_dir):
#     os.mkdir(run_dir)
# save_ROC_dir = run_dir + f"/save_ROC_{now}"
# if not os.path.exists(save_ROC_dir):
#     os.mkdir(save_ROC_dir)

# plt.plot(class_fpr[0], class_tpr[0],'turquoise',label='1: ROC curve of class 1 (area = %0.2f)' % roc_auc[0])
# plt.plot(class_fpr[1], class_tpr[1],'peachpuff',label='2: ROC curve of class 2 (area = %0.2f)' % roc_auc[1])
# plt.plot(class_fpr[2], class_tpr[2],'paleturquoise',label='3: ROC curve of class 3 (area = %0.2f)' % roc_auc[2])
# plt.plot(class_fpr[3], class_tpr[3],'pink',label='4: ROC curve of class 4 (area = %0.2f)' % roc_auc[3])
# plt.plot(class_fpr[4], class_tpr[4],'lightcoral',label='5: ROC curve of class 5 (area = %0.2f)' % roc_auc[4])
# plt.plot(class_fpr[5], class_tpr[5],'peachpuff',label='6: ROC curve of class 6 (area = %0.2f)' % roc_auc[5])
# plt.plot(class_fpr[6], class_tpr[6],'steelblue',label='7: ROC curve of class 7 (area = %0.2f)' % roc_auc[6])
# plt.plot(class_fpr[7], class_tpr[7],'forestgreen',label='8: ROC curve of class 8 (area = %0.2f)' % roc_auc[6])
# plt.plot(class_fpr[8], class_tpr[8],'darkslategray',label='9: ROC curve of class 9 (area = %0.2f)' % roc_auc[6])

# plt.plot([0, 1], [0, 1], 'k--')
# plt.xlim([-0.1, 1.1])
# plt.ylim([-0.1, 1.1])
# plt.xlabel('False Positive Rate')
# plt.ylabel('True Positive Rate')
# plt.title('Receiver operating characteristic of lead')
# plt.legend(loc="lower right")
# plt.savefig(save_ROC_dir)

Epoch: 1


0it [00:00, ?it/s]

1it [00:26, 26.03s/it]

10it [00:26,  1.89s/it]

20it [00:26,  1.30it/s]

...

1013it [00:04, 251.46it/s]

1039it [00:04, 245.52it/s]

1059it [00:04, 236.53it/s]

valid loss: 1.6646815309677774 - valid acc: 39.94334277620396
Best acuracy: 0.45609065155807366 at epoch 0


/media/mountHDD2/linh/git/.env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/media/mountHDD2/linh/git/.env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/media/mountHDD2/linh/git/.env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [31]:
# print(reports.keys())
# print(reports['macro avg'])

# Evaluation